<a href="https://colab.research.google.com/github/SebastianWikt/DS4002-P1/blob/main/Project_1_text_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importance of Word Choice in Education

### Group 5: Neha Channamraju (leader), Sebastian Wiktorowicz, Catherine Young

In [1]:
#ONLY NEED TO RUN THIS ONE TIME
!pip install nltk
!pip install spacy
!pip install scikit-learn
!pip install textblob
!pip install wordcloud
!pip install pandas numpy matplotlib seaborn

#Install spacy extra data
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 13.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!rm -r DS4002-P1/

In [2]:
! git clone https://github.com/SebastianWikt/DS4002-P1.git

Cloning into 'DS4002-P1'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 11), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (25/25), 42.14 KiB | 3.83 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [3]:
from typing_extensions import dataclass_transform
import os # to run file I/O operation
# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import scatter_matrix

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
import re
import string
import datetime
from wordcloud import WordCloud
from textblob import TextBlob
import spacy
from collections import Counter
np.random.seed(42)

# Download necessary NLTK data
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

# Load English tokenizer, tagger, parser, NER, and word vectors from spacy
nlp = spacy.load('en_core_web_sm')

def loadData(file):
  # The column names are adopted from the dataset page on Kaggle
  df = pd.read_csv(file)
  return df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
data = pd.read_csv('/content/DS4002-P1/project_1_dataset.csv')

In [7]:
data = data.rename(columns={"Unnamed: 42": "no_of_words"})

In [8]:
print(data.columns)

Index(['Rank in 2020', 'Rank in 2019', 'Rank in 2018', '3 year average rank',
       'School name', 'Country', 'Audit year', 'Salary today (US$)',
       'Weighted salary (US$)', 'Salary percentage increase',
       'Value for money rank', 'Career progress rank', 'Aims achieved (%)',
       'Careers service rank', 'Employed at three months (%)',
       'Alumni recommend rank', 'Female faculty (%)', 'Female students (%)',
       'Women on board (%)', 'International faculty (%)',
       'International students (%)', 'International board (%)',
       'International mobility rank', 'International course experience rank',
       'Languages', 'Faculty with doctorates (%)', 'FT research rank',
       'CSR rank', 'Unnamed: 28', 'Value statement', 'No. of words', 'Clear',
       'Realistic', 'Shorter', 'Easy to communicate', 'Easy to memorize',
       'Easy to be explained', 'Focused', 'Unique', 'Decision Making guidance',
       'Challenging', 'Mission Statements', 'no_of_words', 'Customers',


In [ ]:
# Import the TextBlob library
from textblob import TextBlob

# Define a function to calculate the polarity and subjectivity of a text
def get_sentiment(text):
	blob = TextBlob(text)
	return blob.sentiment.polarity, blob.sentiment.subjectivity

# Apply the function to the reviews
data['polarity'], data['subjectivity'] = zip(*data['School name'].map(get_sentiment))

# Display the first few rows of the dataframe
data.head()

,Rank in 2020,Rank in 2019,Rank in 2018,3 year average rank,School name,Country,Audit year,Salary today (US$),Weighted salary (US$),Salary percentage increase,...,Socially responsible,Environmentally responsible,Reconciliatory,Enduring,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,polarity,subjectivity
0,1,2,5,3,Harvard Business School,US,2018,"208,023","210,110",110,...,N,N,N,N,"Respect, Honesty, Accountability",3,https://www.hbs.edu/about/Pages/mission.aspx,NaN,0.0,0.0
1,2,4,3,3,University of Pennsylvania: Wharton,US,2018,"217,141","211,543",107,...,Y,N,Y,Y,"Tradition, teamwork and collaboration, prof...",9,https://executiveeducation.wharton.upenn.edu/a...,NaN,0.0,0.0
2,3,1,1,2,Stanford Graduate School of Business,US,2015,"225,589","222,625",117,...,Y,N,Y,Y,"integrity, diversity, respect, freedom of i...",11,https://www.gsb.stanford.edu/experience/about,NaN,0.0,0.0
3,4,3,2,3,Insead,France / Singapore,2020,"182,606","181,277",101,...,Y,N,Y,Y,"Diversity, Independence, Rigour and relevanc...",8,https://www.insead.edu/about/who-we-are,NaN,0.0,0.0
4,5,5,8,6,Ceibs,China,2019,"184,731","185,103",187,...,N,N,N,Y,"Conscientiousness, Innovation and Excellence.",4,https://www.ceibs.edu/introduction-ceibs#:~:te...,NaN,0.0,0.0
